In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re


# def clean_value(value):
#     """달러 기호와 쉼표를 제거하고 숫자 형식으로 변환"""
#     if not value or value == "-":
#         return None

#     value = value.replace("$", "").replace(",", "")

#     try:
#         return float(value)
#     except ValueError:
#         return value


def get_company_name(ticker):
    """티커 심볼로부터 회사 이름을 가져옵니다"""
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}"

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=chrome_options
    )

    try:
        driver.get(url)
        time.sleep(2)  # 페이지 로드 대기

        # 리디렉션된 URL에서 회사 이름 추출
        current_url = driver.current_url
        company_name = current_url.split("/")[-2]

        return company_name
    except Exception as e:
        print(f"회사 이름 추출 오류: {e}")
        return ticker.lower()
    finally:
        driver.quit()


def get_financial_data(ticker, frequency="Q"):
    """Selenium을 사용하여 투자 판단에 필요한 주요 재무제표 데이터만 가져옵니다."""

    # 회사 이름 가져오기
    company_name = get_company_name(ticker)
    print(f"회사 이름: {company_name}")

    # 재무제표 URL (회사명 동적 적용)
    urls = {
        "income_statement": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/income-statement?freq={frequency}",
        "balance_sheet": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/balance-sheet?freq={frequency}",
        "cash_flow": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/cash-flow-statement?freq={frequency}",
        "financial_ratios": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/financial-ratios?freq={frequency}",
    }

    # 투자 판단에 필요한 주요 지표 정의 (정확한 이름으로 수정)
    key_metrics = {
        "income_statement": [
            "Revenue",
            "Gross Profit",
            "Operating Income",
            "Net Income",
            "EPS - Earnings Per Share",
            "Basic EPS",
            "EBITDA",
            "EBIT",
            "Pre-Tax Income",
        ],
        "balance_sheet": [
            "Cash On Hand",
            "Total Current Assets",
            "Total Assets",
            "Total Current Liabilities",
            "Long Term Debt",
            "Total Liabilities",
            "Share Holder Equity",
            "Total Liabilities And Share Holders Equity",
        ],
        "cash_flow": [
            "Cash Flow From Operating Activities",
            "Cash Flow From Investing Activities",
            "Cash Flow From Financial Activities",
            "Net Cash Flow",
            "Free Cash Flow",
        ],
        "financial_ratios": [
            "Current Ratio",
            "Debt/Equity Ratio",
            "Gross Margin",
            "Operating Margin",
            "Net Profit Margin",
            "ROE - Return On Equity",
            "ROA - Return On Assets",
            "Book Value Per Share",
            "Free Cash Flow Per Share",
            "PE Ratio",
            "PB Ratio",
        ],
    }

    # 브라우저 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 화면 없이 실행 (필요시 주석 처리)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    dfs = {}
    latest_data = {}  # 각 티커별 최신 데이터를 저장할 딕셔너리

    # Selenium 웹드라이버 초기화
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=chrome_options
    )

    try:
        for page_type, url in urls.items():
            print(f"\n{ticker}의 {page_type} 데이터 가져오는 중...")
            print(f"URL: {url}")

            driver.get(url)

            # 페이지 로딩 시간 부여 (충분히 길게)
            time.sleep(8)

            # JavaScript를 사용하여 더 정확하게 날짜별 데이터 추출하기
            result = driver.execute_script(
                """
                var result = {headers: [], rows: []};
                var headerCells = document.querySelectorAll('div[role="columnheader"]');
                
                // 헤더 정보 추출 (날짜)
                for (var i = 0; i < headerCells.length; i++) {
                    result.headers.push(headerCells[i].textContent.trim());
                }
                
                var rows = document.querySelectorAll('div[role="row"]');
                
                for (var i = 0; i < rows.length; i++) {
                    var row = rows[i];
                    var cells = row.querySelectorAll('div[role="gridcell"]');
                    
                    if (cells.length === 0) continue;
                    
                    var rowData = {};
                    
                    // 첫 번째 셀은 지표명
                    var metricCell = cells[0];
                    var boldText = metricCell.querySelector('div[style*="font-weight:bold"]');
                    
                    var metricName = boldText ? boldText.textContent.trim() : metricCell.textContent.trim();
                    if (!metricName) continue;
                    
                    rowData["Metric"] = metricName;
                    
                    // 나머지 셀은 값 - 헤더와 정확히 매핑
                    for (var j = 1; j < cells.length && j < result.headers.length; j++) {
                        rowData[result.headers[j]] = cells[j].textContent.trim();
                    }
                    
                    if (Object.keys(rowData).length > 1) {
                        result.rows.push(rowData);
                    }
                }
                
                return result;
            """
            )

            # JavaScript에서 반환된 구조에서 헤더와 행 데이터 추출
            date_headers = result["headers"]
            rows_data = result["rows"]

            print(f"추출된 날짜 헤더: {date_headers}")
            print(f"추출된 행 수: {len(rows_data)}")

            if not date_headers or len(date_headers) < 2:
                print(f"{page_type}에서 날짜 헤더를 찾을 수 없습니다.")
                continue

            # 첫 번째 열은 'Metric' 헤더이므로 제외하고 날짜 헤더만 추출
            # (빈 문자열이 있으면 제외)
            date_headers = [h for h in date_headers[1:] if h.strip()]

            if not date_headers:
                print(f"{page_type}에서 유효한 날짜 헤더를 찾을 수 없습니다.")
                continue

            # 데이터 처리 및 필터링
            all_data_rows = []
            filtered_data_rows = []
            latest_metrics = {}  # 각 지표별 최신 데이터

            for row in rows_data:
                metric_name = row["Metric"]

                if not metric_name or metric_name == "Metric":
                    continue

                # 행 데이터 생성 (모든 날짜)
                row_data = {"Metric": metric_name}

                # 각 날짜 컬럼에 대한 값 매핑
                for date in date_headers:
                    if date in row:
                        row_data[date] = row[date]
                    else:
                        row_data[date] = None

                # 모든 데이터 행 저장
                all_data_rows.append(row_data)

                # 주요 지표 필터링 (정확히 일치하는 지표만 필터링)
                if metric_name in key_metrics[page_type]:
                    filtered_data_rows.append(row_data)

                    # 가장 최신 날짜의 값 저장 (첫 번째 날짜 컬럼)
                    if date_headers and date_headers[0] in row:
                        latest_value = row[date_headers[0]]
                        latest_metrics[metric_name] = latest_value

            # 데이터프레임 생성 (필터링된 데이터 사용)
            if filtered_data_rows:
                df = pd.DataFrame(filtered_data_rows)
                dfs[page_type] = df
                print(
                    f"{page_type} 데이터프레임 생성 완료. 행 수: {len(df)} (필터링된 주요 지표만)"
                )
                print(
                    f"총 지표 수: {len(all_data_rows)}, 필터링된 주요 지표 수: {len(filtered_data_rows)}"
                )

                # try:
                #     # 엑셀 파일로 저장 (openpyxl 필요)
                #     df.to_excel(f"{ticker}_{page_type}_key_metrics.xlsx", index=False)
                #     print(
                #         f"{ticker}_{page_type}_key_metrics.xlsx 파일로 저장되었습니다."
                #     )
                # except ImportError:
                #     print(
                #         "엑셀 저장 실패: openpyxl 모듈이 필요합니다. 'pip install openpyxl'로 설치하세요."
                #     )
                #     # CSV로 대체 저장
                #     df.to_csv(f"{ticker}_{page_type}_key_metrics.csv", index=False)
                #     print(
                #         f"{ticker}_{page_type}_key_metrics.csv 파일로 저장되었습니다."
                #     )

                # 최신 데이터 저장
                latest_data[page_type] = latest_metrics
            else:
                print(f"{page_type}에서 주요 지표 데이터를 찾을 수 없습니다.")

            # 다음 페이지로 가기 전에 잠시 대기
            time.sleep(3)

    except Exception as e:
        print(f"전체 처리 오류: {e}")
        import traceback

        traceback.print_exc()

    finally:
        # 웹드라이버 종료
        driver.quit()

    # 최신 데이터 데이터프레임 생성
    latest_dfs = {}
    for page_type, metrics in latest_data.items():
        # 각 지표와 그 값을 Series로 변환
        latest_df = pd.DataFrame(
            list(metrics.items()), columns=["Metric", "Latest Value"]
        )
        latest_dfs[page_type] = latest_df

        # 파일로 저장
        try:
            latest_df.to_excel(f"{ticker}_{page_type}_latest.xlsx", index=False)
            print(f"{ticker}_{page_type}_latest.xlsx 파일로 저장되었습니다.")
        except ImportError:
            latest_df.to_csv(f"{ticker}_{page_type}_latest.csv", index=False)
            print(f"{ticker}_{page_type}_latest.csv 파일로 저장되었습니다.")

    return dfs, latest_dfs


def main():
    # 분석할 티커 목록
    tickers = ["AAPL"]

    all_data = {}
    all_latest_data = {}

    for ticker in tickers:
        print(f"\n{ticker} 데이터 가져오기 시작:")
        financial_data, latest_data = get_financial_data(ticker)

        if not financial_data:
            print(f"{ticker} 데이터를 가져오지 못했습니다.")
            continue

        all_data[ticker] = financial_data
        all_latest_data[ticker] = latest_data

        # 데이터프레임 확인
        for report_type, df in financial_data.items():
            print(f"\n{ticker} {report_type.upper()} 보고서 주요 지표 (처음 5행):")
            if not df.empty:
                print(df.head())
            else:
                print("데이터가 없습니다.")

        # 최신 데이터 확인
        for report_type, df in latest_data.items():
            print(f"\n{ticker} {report_type.upper()} 최신 데이터:")
            if not df.empty:
                print(df)
            else:
                print("최신 데이터가 없습니다.")

        # 다음 티커로 가기 전에 잠시 대기
        time.sleep(5)

    return all_data, all_latest_data


if __name__ == "__main__":
    data, latest_data = main()

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re


def get_company_name(ticker):
    """티커 심볼로부터 회사 이름을 가져옵니다"""
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}"

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=chrome_options
    )

    try:
        driver.get(url)
        time.sleep(2)  # 페이지 로드 대기

        # 리디렉션된 URL에서 회사 이름 추출
        current_url = driver.current_url
        company_name = current_url.split("/")[-2]

        return company_name
    except Exception as e:
        print(f"회사 이름 추출 오류: {e}")
        return ticker.lower()
    finally:
        driver.quit()


def get_financial_data(ticker, frequency="Q"):
    """Selenium을 사용하여 투자 판단에 필요한 주요 재무제표 데이터만 가져옵니다."""

    # 회사 이름 가져오기
    company_name = get_company_name(ticker)
    print(f"회사 이름: {company_name}")

    # 재무제표 URL (회사명 동적 적용)
    urls = {
        "income_statement": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/income-statement?freq={frequency}",
        "balance_sheet": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/balance-sheet?freq={frequency}",
        "cash_flow": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/cash-flow-statement?freq={frequency}",
        "financial_ratios": f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_name}/financial-ratios?freq={frequency}",
    }

    # 투자 판단에 필요한 주요 지표 정의 (정확한 이름으로 수정)
    key_metrics = {
        "income_statement": [
            "Revenue",
            "Gross Profit",
            "Operating Income",
            "Net Income",
            "EPS - Earnings Per Share",
            "Basic EPS",
            "EBITDA",
            "EBIT",
            "Pre-Tax Income",
        ],
        "balance_sheet": [
            "Cash On Hand",
            "Total Current Assets",
            "Total Assets",
            "Total Current Liabilities",
            "Long Term Debt",
            "Total Liabilities",
            "Share Holder Equity",
            "Total Liabilities And Share Holders Equity",
        ],
        "cash_flow": [
            "Cash Flow From Operating Activities",
            "Cash Flow From Investing Activities",
            "Cash Flow From Financial Activities",
            "Net Cash Flow",
            "Free Cash Flow",
        ],
        "financial_ratios": [
            "Current Ratio",
            "Debt/Equity Ratio",
            "Gross Margin",
            "Operating Margin",
            "Net Profit Margin",
            "ROE - Return On Equity",
            "ROA - Return On Assets",
            "Book Value Per Share",
            "Free Cash Flow Per Share",
            "PE Ratio",
            "PB Ratio",
        ],
    }

    # 브라우저 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 화면 없이 실행 (필요시 주석 처리)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    dfs = {}
    latest_data = {}  # 각 티커별 최신 데이터를 저장할 딕셔너리

    # Selenium 웹드라이버 초기화
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=chrome_options
    )

    try:
        for page_type, url in urls.items():
            print(f"\n{ticker}의 {page_type} 데이터 가져오는 중...")
            print(f"URL: {url}")

            driver.get(url)

            # 페이지 로딩 시간 부여 (충분히 길게)
            time.sleep(8)

            # JavaScript를 사용하여 더 정확하게 날짜별 데이터 추출하기
            result = driver.execute_script(
                """
                var result = {headers: [], rows: []};
                var headerCells = document.querySelectorAll('div[role="columnheader"]');
                
                // 헤더 정보 추출 (날짜)
                for (var i = 0; i < headerCells.length; i++) {
                    result.headers.push(headerCells[i].textContent.trim());
                }
                
                var rows = document.querySelectorAll('div[role="row"]');
                
                for (var i = 0; i < rows.length; i++) {
                    var row = rows[i];
                    var cells = row.querySelectorAll('div[role="gridcell"]');
                    
                    if (cells.length === 0) continue;
                    
                    var rowData = {};
                    
                    // 첫 번째 셀은 지표명
                    var metricCell = cells[0];
                    var boldText = metricCell.querySelector('div[style*="font-weight:bold"]');
                    
                    var metricName = boldText ? boldText.textContent.trim() : metricCell.textContent.trim();
                    if (!metricName) continue;
                    
                    rowData["Metric"] = metricName;
                    
                    // 나머지 셀은 값 - 헤더와 정확히 매핑
                    for (var j = 1; j < cells.length && j < result.headers.length; j++) {
                        rowData[result.headers[j]] = cells[j].textContent.trim();
                    }
                    
                    if (Object.keys(rowData).length > 1) {
                        result.rows.push(rowData);
                    }
                }
                
                return result;
            """
            )

            # JavaScript에서 반환된 구조에서 헤더와 행 데이터 추출
            date_headers = result["headers"]
            rows_data = result["rows"]

            print(f"추출된 날짜 헤더: {date_headers}")
            print(f"추출된 행 수: {len(rows_data)}")

            if not date_headers or len(date_headers) < 2:
                print(f"{page_type}에서 날짜 헤더를 찾을 수 없습니다.")
                continue

            # 첫 번째 열은 'Metric' 헤더이므로 제외하고 날짜 헤더만 추출
            # (빈 문자열이 있으면 제외)
            date_headers = [h for h in date_headers[1:] if h.strip()]

            if not date_headers:
                print(f"{page_type}에서 유효한 날짜 헤더를 찾을 수 없습니다.")
                continue

            # 데이터 처리 및 필터링
            all_data_rows = []
            filtered_data_rows = []
            latest_metrics = {}  # 각 지표별 최신 데이터

            for row in rows_data:
                metric_name = row["Metric"]

                if not metric_name or metric_name == "Metric":
                    continue

                # 행 데이터 생성 (모든 날짜)
                row_data = {"Metric": metric_name}

                # 각 날짜 컬럼에 대한 값 매핑
                for date in date_headers:
                    if date in row:
                        row_data[date] = row[date]
                    else:
                        row_data[date] = None

                # 모든 데이터 행 저장
                all_data_rows.append(row_data)

                # 주요 지표 필터링 (정확히 일치하는 지표만 필터링)
                if metric_name in key_metrics[page_type]:
                    filtered_data_rows.append(row_data)

                    # 가장 최신 날짜의 값 저장 (첫 번째 날짜 컬럼)
                    if date_headers and date_headers[0] in row:
                        latest_value = row[date_headers[0]]
                        latest_metrics[metric_name] = latest_value

            # 데이터프레임 생성 (필터링된 데이터 사용)
            if filtered_data_rows:
                df = pd.DataFrame(filtered_data_rows)
                dfs[page_type] = df
                print(
                    f"{page_type} 데이터프레임 생성 완료. 행 수: {len(df)} (필터링된 주요 지표만)"
                )
                print(
                    f"총 지표 수: {len(all_data_rows)}, 필터링된 주요 지표 수: {len(filtered_data_rows)}"
                )

                # 최신 데이터 저장
                latest_data[page_type] = latest_metrics
            else:
                print(f"{page_type}에서 주요 지표 데이터를 찾을 수 없습니다.")

            # 다음 페이지로 가기 전에 잠시 대기
            time.sleep(3)

    except Exception as e:
        print(f"전체 처리 오류: {e}")
        import traceback

        traceback.print_exc()

    finally:
        # 웹드라이버 종료
        driver.quit()

    # 최신 데이터 데이터프레임 생성
    latest_dfs = {}
    for page_type, metrics in latest_data.items():
        # 각 지표와 그 값을 Series로 변환
        latest_df = pd.DataFrame(
            list(metrics.items()), columns=["Metric", "Latest Value"]
        )

        # 기업 정보 추가
        latest_df["Ticker"] = ticker
        latest_df["Company"] = company_name

        latest_dfs[page_type] = latest_df

    return dfs, latest_dfs


def main():
    # 분석할 티커 목록
    tickers = ["AAPL", "NVDA", "TSLA"]  # 원하는 티커 목록으로 변경 가능

    # 보고서 유형별로 모든 기업 데이터를 저장할 딕셔너리
    combined_latest_data = {
        "income_statement": [],
        "balance_sheet": [],
        "cash_flow": [],
        "financial_ratios": [],
    }

    # 모든 기업의 데이터를 원본 형태로 저장할 딕셔너리
    all_companies_data = {}

    for ticker in tickers:
        print(f"\n{ticker} 데이터 가져오기 시작:")
        financial_data, latest_data = get_financial_data(ticker)

        if not financial_data:
            print(f"{ticker} 데이터를 가져오지 못했습니다.")
            continue

        # 티커별 데이터 저장
        all_companies_data[ticker] = latest_data

        # 최신 데이터를 보고서 유형별로 병합
        for report_type, df in latest_data.items():
            if not df.empty:
                combined_latest_data[report_type].append(df)
                print(f"{ticker} {report_type} 데이터 병합 완료. 행 수: {len(df)}")
            else:
                print(f"{ticker} {report_type} 데이터가 없습니다.")

        # 다음 티커로 가기 전에 잠시 대기
        time.sleep(3)

    # 모든 보고서 유형의 모든 지표를 하나의 통합 데이터프레임으로 생성
    complete_integrated_data = []

    for ticker, report_types in all_companies_data.items():
        for report_type, df in report_types.items():
            if not df.empty:
                # 보고서 유형 정보 추가
                df["Report_Type"] = report_type
                complete_integrated_data.append(df)

    # 모든 데이터 통합 - 기업을 행으로, 지표를 열로
    if complete_integrated_data:
        all_metrics_df = pd.concat(complete_integrated_data, ignore_index=True)

        # 피벗 테이블 생성 (기업을 행으로, 지표와 보고서 유형 조합을 열로)
        complete_pivot_df = all_metrics_df.pivot_table(
            index=["Ticker", "Company"],
            columns=["Report_Type", "Metric"],
            values="Latest Value",
            aggfunc="first",
        )

        # 인덱스 형식 변경 - 티커 (회사명)
        complete_pivot_df.index = [
            f"{ticker} ({company})" for ticker, company in complete_pivot_df.index
        ]

        # 통합 데이터 저장
        complete_pivot_df.to_excel("all_companies_complete_data.xlsx")
        print(
            "\n모든 지표의 통합 데이터를 all_companies_complete_data.xlsx 파일로 저장했습니다."
        )

    # 모든 기업의 데이터를 보고서 유형별로 처리하고 피벗 테이블 생성
    all_pivot_dfs = {}  # 모든 보고서 유형의 피벗 테이블을 저장

    for report_type, dfs_list in combined_latest_data.items():
        if dfs_list:
            # 모든 기업의 데이터 하나로 합치기
            combined_df = pd.concat(dfs_list, ignore_index=True)

            # 피벗 테이블 생성 - 기업을 행으로, 지표를 열로 변경
            pivot_df = combined_df.pivot_table(
                index="Ticker",
                columns="Metric",
                values="Latest Value",
                aggfunc="first",  # 중복값이 있을 경우 첫 번째 값 사용
            )

            # 회사 이름 정보 추가
            company_info_df = (
                combined_df[["Ticker", "Company"]].drop_duplicates().set_index("Ticker")
            )

            # 인덱스에 회사 이름 추가 (티커 (회사명) 형식)
            pivot_df.index = [
                f"{ticker} ({company_info_df.loc[ticker, 'Company']})"
                for ticker in pivot_df.index
            ]

            # 피벗 테이블 저장 - 개별 보고서 파일
            pivot_df.to_excel(f"all_companies_{report_type}.xlsx")
            print(
                f"\n모든 기업의 {report_type} 데이터를 all_companies_{report_type}.xlsx 파일로 저장했습니다."
            )

            # 통합 파일을 위해 피벗 테이블 저장
            all_pivot_dfs[report_type] = pivot_df

    # 통합 파일 생성 - 모든 보고서 유형의 데이터를 하나의 엑셀 파일에 시트별로 저장
    if all_pivot_dfs:
        try:
            with pd.ExcelWriter(
                "all_companies_integrated.xlsx", engine="openpyxl"
            ) as writer:
                for report_type, pivot_df in all_pivot_dfs.items():
                    # 각 보고서 유형을 별도의 시트로 저장
                    print(
                        f"통합 파일에 {report_type} 시트 추가 중... 데이터 크기: {pivot_df.shape}"
                    )
                    pivot_df.to_excel(writer, sheet_name=report_type)

            print(
                "\n모든 기업의 통합 데이터를 all_companies_integrated.xlsx 파일로 저장했습니다."
            )
        except Exception as e:
            print(f"\n통합 파일 생성 중 오류 발생: {e}")

            # 대안 방법: 각 보고서 유형을 별도의 시트 이름으로 CSV 파일로 저장
            for report_type, pivot_df in all_pivot_dfs.items():
                try:
                    # CSV 파일로 저장
                    pivot_df.to_csv(f"all_companies_integrated_{report_type}.csv")
                    print(
                        f"{report_type} 데이터를 all_companies_integrated_{report_type}.csv 파일로 저장했습니다."
                    )
                except Exception as sub_e:
                    print(f"{report_type} CSV 저장 중 오류: {sub_e}")

    # 파일 목록 출력
    print("\n생성된 파일 목록:")
    print(
        "1. all_companies_income_statement.xlsx - 손익계산서 지표 (기업이 행, 지표가 열)"
    )
    print(
        "2. all_companies_balance_sheet.xlsx - 대차대조표 지표 (기업이 행, 지표가 열)"
    )
    print("3. all_companies_cash_flow.xlsx - 현금흐름표 지표 (기업이 행, 지표가 열)")
    print(
        "4. all_companies_financial_ratios.xlsx - 재무비율 지표 (기업이 행, 지표가 열)"
    )
    print(
        "5. all_companies_integrated.xlsx - 위 모든 데이터를 시트별로 통합한 파일 (기업이 행, 지표가 열)"
    )
    print(
        "6. all_companies_complete_data.xlsx - 모든 지표를 하나의 테이블로 통합 (기업이 행, 보고서유형+지표가 열)"
    )

    return combined_latest_data


if __name__ == "__main__":
    combined_data = main()